### 第6週、2日目

独自のMCPサーバーとMCPクライアントを作成して使用しようとしています！

それは非常に簡単ですが、それは非常に単純ではありません。 

MCPに関する興奮は、他のMCPサーバーを共有して使用するのがどれほど簡単かについてです。

独自のMCPサーバーを作成するには、少しの作業が含まれます。

主に勤勉なエンジニアリングチームによって作成されたいくつかのPythonコードを確認しましょう。: accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
# コレは（account.py中に実装されている）以前作成したモノに類似の
# 株式取引シミュレーション用のアカウント管理システムのクラス
from accounts import Account

In [3]:
# Accountクラスを使って「Ed」のアカウントを取得した結果
account = Account.get("Ed")
account

Account(name='ed', balance=9741.484, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 86.172 each.], portfolio_value_time_series=[('2025-09-14 13:06:58', 9978.484), ('2025-09-14 13:06:58', 9846.484)])

In [4]:
# Accountクラスを使って Edが AMZN 株を3株購入した結果
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9648.298, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-09-14 13:06:58", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-09-14 16:52:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-14 13:06:58", 9978.484], ["2025-09-14 13:06:58", 9846.484], ["2025-09-14 16:52:30", 9924.298]], "total_portfolio_value": 9924.298, "total_profit_loss": -75.70199999999932}'

In [5]:
# Accountクラスを使って Ed の最新のアカウント状態をレポート出力
account.report()

'{"name": "ed", "balance": 9648.298, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-09-14 13:06:58", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-09-14 16:52:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-14 13:06:58", 9978.484], ["2025-09-14 13:06:58", 9846.484], ["2025-09-14 16:52:30", 9924.298], ["2025-09-14 16:52:31", 10116.298]], "total_portfolio_value": 10116.298, "total_profit_loss": 116.29800000000068}'

In [6]:
# Accountクラスを使って Ed のアカウントで行われた取引の一覧そ出漁
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 86.172,
  'timestamp': '2025-09-14 13:06:58',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 31.062,
  'timestamp': '2025-09-14 16:52:30',
  'rationale': 'Because this bookstore website looks promising'}]

### 次に、MCPサーバーを書き、直接使用します！

In [7]:
# accounts_server.py によって accounts.py をMCPサーバーとして使用できるようにした。
# 以下は、この accounts_server.py を起動し、STDIOモードのフェッチツールを一覧化する。

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [8]:
# エージェントにMCPのフェッチツールを設定し、口座情報の問い合わせを実行

# クライアントのアカウントを管理し、アカウントに関する質問に答えることができます。
instructions = "You are able to manage an account for a client, and answer questions about the account."
# 私の名前は Ed で、口座名義も Ed です。残高と保有資産はいくらですか？
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
# エージェントが使用するLLMのモデル
model = "gpt-4.1-mini"

# MCPサーバーを使う処理は async with MCPServerStdio の中で行う必要がある。
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your current cash balance is $9,648.30. You hold 6 shares of Amazon (AMZN) stock in your account. If you need any further details or want to make transactions, just let me know!

### それでは、独自のMCPクライアントを構築しましょう

In [9]:
# accounts_client.py によって accounts.py のツールやリソースにアクセスするためのクライアント関数群を定義
# 以下は、accounts_server.py（サーバ）に非同期的に接続する accounts_client.py（クライアント）の実装
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

# mcp_tools [Tool(name=...
mcp_tools = await list_accounts_tools()
print(mcp_tools)

# openai_tools [FunctionTool(name=...
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [10]:
# 私の名前は Ed です。口座名義も Ed です。残高はいくらですか？
request = "My name is Ed and my account is under the name Ed. What's my balance?"

# エージェント・ツールとして使う
with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your current account balance is $9,648.30. Is there anything specific you would like to do with your account or any other information you need?

In [11]:
# サーバー上のリソースを読込関数（アカウント情報）
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9648.298, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-09-14 13:06:58", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-09-14 16:52:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-14 13:06:58", 9978.484], ["2025-09-14 13:06:58", 9846.484], ["2025-09-14 16:52:30", 9924.298], ["2025-09-14 16:52:31", 10116.298], ["2025-09-14 16:53:54", 10230.298]], "total_portfolio_value": 10230.298, "total_profit_loss": 230.29800000000068}


In [12]:
# Account直で呼び出してレポート（上記と比較）
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9648.298, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-09-14 13:06:58", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-09-14 16:52:30", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-14 13:06:58", 9978.484], ["2025-09-14 13:06:58", 9846.484], ["2025-09-14 16:52:30", 9924.298], ["2025-09-14 16:52:31", 10116.298], ["2025-09-14 16:53:54", 10230.298], ["2025-09-14 16:53:55", 10008.298]], "total_portfolio_value": 10008.298, "total_profit_loss": 8.298000000000684}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">演習</h2>
            <span style="color:#ff7800;">
                独自のMCPサーバーを作成しましょう！現在の日付を返す簡単な関数を作成し、それをツールとして公開して、エージェントが今日の日付を教えられるようにします。<br/>より難しいオプション演習：MCPクライアントを作成し、ネイティブのOpenAI呼び出し（エージェントSDKなし）を使用して、クライアント経由でツールを使用します。
            </span>
        </td>
    </tr>
</table>